In [1]:
import pandas as pd
import numpy as np
import json
import re

In [2]:
pay = pd.read_csv('../data/ceo data pay/ceo_data_pay_merged_r3000.csv')

In [3]:
pay = pay.drop(columns=['Unnamed: 0','pay_ratio'])

In [4]:
pay['median_worker_pay'] = pay['median_worker_pay'].str.replace(',','').str.replace('$','')
pay['salary'] = pay['salary'].str.replace(',','').str.replace('$','')
pay['industry'] = pay['industry'].str.replace('%20',' ')
pay['company_name'] = pay['company_name'].str.replace(',','')

C:\Users\jorge\AppData\Local\Temp\ipykernel_17296\2338073337.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  pay['median_worker_pay'] = pay['median_worker_pay'].str.replace(',','').str.replace('$','')
C:\Users\jorge\AppData\Local\Temp\ipykernel_17296\2338073337.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  pay['salary'] = pay['salary'].str.replace(',','').str.replace('$','')


In [5]:
pay['salary'] = pay['salary'].astype(int)
pay['median_worker_pay'] = pay['median_worker_pay'].astype(int)

In [6]:

pay.drop(index=pay[(pay['salary'] == 0) | (pay['median_worker_pay'] == 0)].index,inplace=True)

In [10]:
pay['ratio'] = round(pay['salary'] / pay['median_worker_pay'])
pay['ratio'] = pay['ratio'].astype(int)

In [12]:
pay.to_csv('../data/ceo data pay/clean.csv',index=False)

In [98]:
lays = pd.read_csv('../data/tech layoffs/tech_layoffs.csv')

In [99]:
lays = lays.drop(columns=['additional_notes'])

In [100]:
def new_col(col,*args):
    z = lays['industry'].apply(lambda x : x.lower())
    lays[col] = z.str.contains(col.lower())
    for arg in args:
        lays[col]+= z.str.contains(arg.lower())

In [101]:
new_col('Fintech','finance','financial','payment')
new_col('Real estate','proptech')
new_col('SaaS','software','crm')
new_col('Ecommerce','e-commerce','sales','retail')
new_col('Logistics','shipping','transport')
new_col('Energy','solar','renewable')
new_col('Marketing','Advertising','advert','adtech')
new_col('Food','drink','beverage')
new_col('Health','wellness','beauty')
new_col('Crypto','blockchain','nft')
new_col('Journalism','news','media','sport')
new_col('Analytics')
new_col('Artificial Intelligence','ai','machine','predictive')
new_col('Security','surveillance')
new_col('Cloud')
new_col('IT','information technology','productivity','HR','human','customer service')
new_col('Insurance','insur')
new_col('Video Games','game','gaming')

In [102]:
newtable = lays.copy()
newtable = newtable.drop(columns = ['total_layoffs','impacted_workforce_percentage','reported_date',
                          'industry','headquarter_location','sources','status'])

Index(['company', 'Fintech', 'Real estate', 'SaaS', 'Ecommerce', 'Logistics',
       'Energy', 'Marketing', 'Food', 'Health', 'Crypto', 'Journalism',
       'Analytics', 'Artificial Intelligence', 'Security', 'Cloud', 'IT',
       'Insurance', 'Video Games'],
      dtype='object')